# Основы глубинного обучения, майнор ИАД

## Домашнее задание 1. Введение в PyTorch. Полносвязные нейронные сети.

### Общая информация

Дата выдачи: 20.09.2022

Мягкий дедлайн: 23:59MSK 04.10.2022

Жесткий дедлайн: 23:59MSK 10.10.2022

### Оценивание и штрафы
Максимально допустимая оценка за работу — 10 баллов. За каждый день просрочки снимается 1 балл. Сдавать задание после жёсткого дедлайна сдачи нельзя.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.
Также оценка может быть снижена за плохо читаемый код и плохо оформленные графики. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

### О задании

В этом задании вам предстоит предсказывать год выпуска песни (**задача регрессии**) по некоторым звуковым признакам: [данные](https://archive.ics.uci.edu/ml/datasets/yearpredictionmsd). В ячейках ниже находится код для загрузки данных. Обратите внимание, что обучающая и тестовая выборки располагаются в одном файле, поэтому НЕ меняйте ячейку, в которой производится деление данных.

In [ ]:
import torch
from torch import nn
import pandas as pd
import numpy as np
import random
import warnings
from tqdm.auto import tqdm
warnings.filterwarnings("ignore")

In [ ]:
# %conda install wget

In [ ]:
!wget -O data.txt.zip https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip

--2022-10-03 20:39:06--  https://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211011981 (201M) [application/x-httpd-php]
Saving to: ‘data.txt.zip’

data.txt.zip        100%[===================>] 201.24M  42.3MB/s    in 5.2s    

2022-10-03 20:39:12 (38.6 MB/s) - ‘data.txt.zip’ saved [211011981/211011981]



In [ ]:
df = pd.read_csv('data.txt.zip', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,81,82,83,84,85,86,87,88,89,90
0,2001,49.94357,21.47114,73.07750,8.74861,-17.40628,-13.09905,-25.01202,-12.23257,7.83089,...,13.01620,-54.40548,58.99367,15.37344,1.11144,-23.08793,68.40795,-1.82223,-27.46348,2.26327
1,2001,48.73215,18.42930,70.32679,12.94636,-10.32437,-24.83777,8.76630,-0.92019,18.76548,...,5.66812,-19.68073,33.04964,42.87836,-9.90378,-32.22788,70.49388,12.04941,58.43453,26.92061
2,2001,50.95714,31.85602,55.81851,13.41693,-6.57898,-18.54940,-3.27872,-2.35035,16.07017,...,3.03800,26.05866,-50.92779,10.93792,-0.07568,43.20130,-115.00698,-0.05859,39.67068,-0.66345
3,2001,48.24750,-1.89837,36.29772,2.58776,0.97170,-26.21683,5.05097,-10.34124,3.55005,...,34.57337,-171.70734,-16.96705,-46.67617,-12.51516,82.58061,-72.08993,9.90558,199.62971,18.85382
4,2001,50.97020,42.20998,67.09964,8.46791,-15.85279,-16.81409,-12.48207,-9.37636,12.63699,...,9.92661,-55.95724,64.92712,-17.72522,-1.49237,-7.50035,51.76631,7.88713,55.66926,28.74903


In [ ]:
X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

train_size = 463715
X_train = X[:train_size, :]
y_train = y[:train_size]
X_test = X[train_size:, :]
y_test = y[train_size:]

## Задание 0. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Мы будем использовать RMSE как метрику качества. Для самого первого бейзлайна обучите `Ridge` регрессию из `sklearn`. Кроме того, посчитайте качество при наилучшем константном прогнозе.

In [ ]:
#Нормируем данные
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [ ]:
#Обучаем Ridge и оцениваем качество по RMSE
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

ridge = Ridge().fit(X_train,y_train)
print ("Train: {}".format(mean_squared_error(y_train, ridge.predict(X_train), squared=False)))
print ("Test: {}".format(mean_squared_error(y_test, ridge.predict(X_test), squared=False)))

Train: 9.552822967970322
Test: 9.510160820470437


Оптимальное решение в задаче минимизации MSE и RMSE в классе констант – среднее арифметическое. https://alexanderdyakonov.files.wordpress.com/2018/10/book_08_metrics_12_blog1.pdf стр. 6

In [ ]:
print ("Test: {}".format(mean_squared_error(y_test, np.array([np.mean(y_train)] * len(y_test)), squared=False)))

Test: 10.85246390513634


Можно заметить, что модель, обученная с помощью Ridge регрессии, лишь немного лучше константного прогноза. Пойдем в графы вычислений, чтоб наша модель научилась предсказывать таргет более точно.

## Задание 1. (максимум 10 баллов)

Реализуйте обучение и тестирование нейронной сети для предоставленного вам набора данных. Соотношение между полученным значением метрики на тестовой выборке и баллами за задание следующее:

- $\text{RMSE} \le 9.00 $ &mdash; 4 балла
- $\text{RMSE} \le 8.90 $ &mdash; 6 баллов
- $\text{RMSE} \le 8.80 $ &mdash; 8 баллов
- $\text{RMSE} \le 8.75 $ &mdash; 10 баллов

Есть несколько правил, которых вам нужно придерживаться:

- Весь пайплайн обучения должен быть написан на PyTorch. При этом вы можете пользоваться другими библиотеками (`numpy`, `sklearn` и пр.), но только для обработки данных. То есть как угодно трансформировать данные и считать метрики с помощью этих библиотек можно, а импортировать модели из `sklearn` и выбивать с их помощью требуемое качество &mdash; нельзя. Также нельзя пользоваться библиотеками, для которых сам PyTorch является зависимостью.

- Мы никак не ограничиваем ваш выбор архитектуры модели, но скорее всего вам будет достаточно полносвязной нейронной сети.

- Для обучения запрещается использовать какие-либо иные данные, кроме обучающей выборки.

- Ансамблирование моделей запрещено.

### Полезные советы:

- Очень вряд ли, что у вас с первого раза получится выбить качество на 10 баллов, поэтому пробуйте разные архитектуры, оптимизаторы и значения гиперпараметров. В идеале при запуске каждого нового эксперимента вы должны менять что-то одно, чтобы точно знать, как этот фактор влияет на качество.

- Не забудьте, что для улучшения качества модели вам поможет **нормировка таргета**.

- Тот факт, что мы занимаемся глубинным обучением, не означает, что стоит забывать про приемы, использующиеся в классическом машинном обучении. Так что обязательно проводите исследовательский анализ данных, отрисовывайте нужные графики и не забывайте про масштабирование и подбор гиперпараметров.

- Вы наверняка столкнетесь с тем, что ваша нейронная сеть будет сильно переобучаться. Для нейросетей существуют специальные методы регуляризации, например, dropout ([статья](https://jmlr.org/papers/volume15/srivastava14a/srivastava14a.pdf)) и weight decay ([блогпост](https://towardsdatascience.com/weight-decay-l2-regularization-90a9e17713cd)). Они, разумеется, реализованы в PyTorch. Попробуйте поэкспериментировать с ними.

- Если вы чего-то не знаете, не гнушайтесь гуглить. В интернете очень много полезной информации, туториалов и советов по глубинному обучению в целом и по PyTorch в частности. Но не забывайте, что за скатанный код без ссылки на источник придется ответить по всей строгости!

- Если вы сразу реализуете обучение на GPU, то у вас будет больше времени на эксперименты, так как любые вычисления будут работать быстрее. Google Colab предоставляет несколько GPU-часов (обычно около 8-10) в сутки бесплатно.

- Чтобы отладить код, можете обучаться на небольшой части данных или даже на одном батче. Если лосс на обучающей выборке не падает, то что-то точно идет не так!

- Пользуйтесь утилитами, которые вам предоставляет PyTorch (например, Dataset и Dataloader). Их специально разработали для упрощения разработки пайплайна обучения.

- Скорее всего вы захотите отслеживать прогресс обучения. Для создания прогресс-баров есть удобная библиотека `tqdm`.

- Быть может, вы захотите, чтобы графики рисовались прямо во время обучения. Можете воспользоваться функцией [clear_output](http://ipython.org/ipython-doc/dev/api/generated/IPython.display.html#IPython.display.clear_output), чтобы удалять старый график и рисовать новый на его месте.

**ОБЯЗАТЕЛЬНО** рисуйте графики зависимости лосса/метрики на обучающей и тестовой выборках в зависимости от времени обучения. Если обучение занимает относительно небольшое число эпох, то лучше рисовать зависимость от номера шага обучения, если же эпох больше, то рисуйте зависимость по эпохам. Если проверяющий не увидит такого графика для вашей лучшей модели, то он в праве снизить баллы за задание.

**ВАЖНО!** Ваше решение должно быть воспроизводимым. Если это не так, то проверяющий имеет право снизить баллы за задание. Чтобы зафиксировать random seed, воспользуйтесь функцией из ячейки ниже.



In [ ]:
def set_random_seed(seed):
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(123456)

Вы можете придерживаться любой адекватной струкуры кода, но мы советуем воспользоваться следующими сигнатурами функций. Лучше всего, если вы проверите ваши предсказания ассертом: так вы убережете себя от разных косяков, например, что вектор предсказаний состоит из всего одного числа. В любом случае, внимательно следите за тем, для каких тензоров вы считаете метрику RMSE. При случайном или намеренном введении в заблуждение проверяющие очень сильно разозлятся.

In [ ]:
#нормировка таргета
scaler_y = StandardScaler()
y_train = scaler_y.fit_transform(y_train)
y_test = scaler_y.transform(y_test)

In [ ]:
#позаимствовала https://discuss.pytorch.org/t/rmse-loss-function/16540
class RMSELoss(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps
        
    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss

In [ ]:
class ModelBaseline(nn.Module):
    def __init__(self):
        super(ModelBaseline, self).__init__()
        
        self.Linear1 = nn.Sequential(
            nn.Linear(90, 300),
            nn.Dropout(0.4),
            nn.PReLU(),
            nn.BatchNorm1d(300)
            )
        self.Linear2 = nn.Sequential(
            nn.Linear(300, 200),
            nn.Dropout(0.1),
            nn.PReLU(),
            nn.BatchNorm1d(200)
            )
        self.Linear3 = nn.Sequential(
            nn.Linear(200, 100),
            nn.Dropout(0.1),
            nn.PReLU(),
            nn.BatchNorm1d(100)
            )
        self.predictor = nn.Sequential(
            nn.Linear(100, 1)
            )
        self.net = nn.Sequential(
            self.Linear1,
            self.Linear2,
            self.Linear3,
            self.predictor
            )

    def forward(self, x):
        return self.net(x)

In [ ]:
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)

train_set = torch.utils.data.TensorDataset(X_train, y_train)                  
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1024)        

X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

test_set = torch.utils.data.TensorDataset(X_test, y_test)                                        
test_loader = torch.utils.data.DataLoader(test_set, batch_size=256)   

model = ModelBaseline()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)                     
criterion = RMSELoss()                 
device = "cpu"

In [ ]:
def train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=5):
    '''
    params:
        model - torch.nn.Module to be fitted
        optimizer - model optimizer
        criterion - loss function from torch.nn
        train_loader - torch.utils.data.Dataloader with train set
        test_loader - torch.utils.data.Dataloader with test set
                      (if you wish to validate during training)
    '''
    train_losses, test_rmse = [], []
    model = model.to(device)

    for epoch in range(1, n_epochs+1):
        # тренировка
        train_epoch_loss = []
        model.train()
        for x, y in tqdm(train_loader, desc=f"Training, epoch {epoch}", leave=False):
            x, y = x.to(device), y.to(device)

            y_pred = model(x).to(device)
            loss = criterion(y_pred, y)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            with torch.no_grad():
                rmse_train = mean_squared_error(scaler_y.inverse_transform(y_pred), scaler_y.inverse_transform(y), squared=False)
                train_epoch_loss.append(rmse_train)

        loss_train = np.mean(train_epoch_loss)
        train_losses.append(loss_train) 

        pred_test = test(model, criterion, test_loader, device)
        rmse = mean_squared_error(scaler_y.inverse_transform(pred_test), scaler_y.inverse_transform(y_test), squared=False)
        test_rmse.append(rmse)

        wandb.log({"train loss": float(loss_train),
                   "test rmse": rmse})
        
        print(f"Epoch: {epoch}, loss: {loss_train}, rmse: {rmse}")

def test(model, criterion, test_loader, device):
    '''
    params:
        model - torch.nn.Module to be evaluated on test set
        criterion - loss function from torch.nn
        test_loader - torch.utils.data.Dataloader with test set
    ----------
    returns:
        predicts - torch.tensor with shape (len(test_loader.dataset), ),
                   which contains predictions for test objects
    '''
    model.eval()
    with torch.no_grad():
      predicts = torch.empty(0)

      for x_test, y_test in tqdm(test_loader):
          x_test, y_test = x_test.to(device), y_test.to(device)
          y_pred = model(x_test).to(device)
          predicts = torch.cat((predicts, y_pred), 0)
          
    return predicts

##Эксперименты

In [ ]:
!pip install wandb --upgrade --quiet
import wandb
wandb.login()
wandb.init(project="hw1_1")

     |████████████████████████████████| 1.8 MB 26.0 MB/s 
     |████████████████████████████████| 162 kB 66.3 MB/s 
     |████████████████████████████████| 181 kB 54.9 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 162 kB 57.3 MB/s 
     |████████████████████████████████| 158 kB 71.5 MB/s 
     |████████████████████████████████| 157 kB 48.8 MB/s 
     |████████████████████████████████| 157 kB 68.5 MB/s 
     |████████████████████████████████| 157 kB 41.0 MB/s 
     |████████████████████████████████| 157 kB 90.5 MB/s 
     |████████████████████████████████| 157 kB 87.1 MB/s 
     |████████████████████████████████| 157 kB 59.9 MB/s 
     |████████████████████████████████| 157 kB 76.5 MB/s 
     |████████████████████████████████| 156 kB 54.6 MB/s 


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: liz04ka. Use `wandb login --relogin` to force relogin


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_1", 
      config={
      "batch_size train": 1024,
      "optimizer": "SGD",
      "learning_rate": 0.01,
      "hidden": "90/10/1",
      "rectifier": "ReLU",
      "epochs": 5,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=5)
wandb.finish()

Training, epoch 1:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 10.355486971294045, rmse: 9.958621506344306


Training, epoch 2:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 4, loss: 9.255779789295822, rmse: 9.330049967596066


Training, epoch 5:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 5, loss: 9.194283633621028, rmse: 9.286747113214734


test rmse,█▄▂▁▁
train loss,█▄▂▁▁
test rmse,9.28675
train loss,9.19428


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_2", 
      config={
      "batch_size train": 1024,
      "optimizer": "SGD",
      "learning_rate": 0.001,
      "hidden": "90/10/1",
      "rectifier": "ReLU",
      "epochs": 5,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=5)
wandb.finish()

Training, epoch 1:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.155772013035728, rmse: 9.258478783191562


Training, epoch 2:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 4, loss: 9.094077847371125, rmse: 9.213681233508073


Training, epoch 5:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 5, loss: 9.080967903520094, rmse: 9.20427298539174


test rmse,█▆▄▂▁
train loss,█▆▄▂▁
test rmse,9.20427
train loss,9.08097


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_3", 
      config={
      "batch_size train": 1024,
      "optimizer": "SGD",
      "learning_rate": 0.001,
      "hidden": "90/100/40/1",
      "rectifier": "ReLU",
      "epochs": 5,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=5)
wandb.finish()

Training, epoch 1:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 10.76558595341297, rmse: 10.823586003182289


Training, epoch 2:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 4, loss: 10.6476065654595, rmse: 10.707605060751039


Training, epoch 5:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 5, loss: 10.60608957828094, rmse: 10.66508882906144


test rmse,█▆▅▃▁
train loss,█▆▅▃▁
test rmse,10.66509
train loss,10.60609


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_4", 
      config={
      "batch_size train": 1024,
      "optimizer": "SGD",
      "learning_rate": 0.001,
      "hidden": "90/100/40/1",
      "BatchNorm1d": "True",
      "rectifier": "ReLU",
      "epochs": 5,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=5)
wandb.finish()

Training, epoch 1:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 11.1922914276338, rmse: 10.681125687784444


Training, epoch 2:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 4, loss: 9.957915120144161, rmse: 9.96187170701382


Training, epoch 5:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 5, loss: 9.839843447306828, rmse: 9.8555690209977


test rmse,█▅▃▂▁
train loss,█▄▂▂▁
test rmse,9.85557
train loss,9.83984


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_5", 
      config={
      "batch_size train": 1024,
      "optimizer": "Adam",
      "learning_rate": 0.001,
      "hidden": "90/100/40/1",
      "BatchNorm1d": "True",
      "rectifier": "ReLU",
      "epochs": 5,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=5)
wandb.finish()

Training, epoch 1:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.302860990493933, rmse: 9.039854797654476


Training, epoch 2:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 4, loss: 8.802883489186005, rmse: 8.886605107527378


Training, epoch 5:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 5, loss: 8.76395959511067, rmse: 8.873064556662747


test rmse,█▄▃▂▁
train loss,█▃▂▂▁
test rmse,8.87306
train loss,8.76396


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_6", 
      config={
      "batch_size train": 512,
      "optimizer": "Adam",
      "learning_rate": 0.001,
      "hidden": "90/100/40/1",
      "BatchNorm1d": "True",
      "rectifier": "ReLU",
      "epochs": 5,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=5)
wandb.finish()

Training, epoch 1:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.236741407048834, rmse: 9.031418405579805


Training, epoch 2:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 4, loss: 8.807193529089671, rmse: 8.903664113901975


Training, epoch 5:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 5, loss: 8.770706693171524, rmse: 8.893224118537317


test rmse,█▄▂▂▁
train loss,█▃▂▂▁
test rmse,8.89322
train loss,8.77071


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_7", 
      config={
      "batch_size train": 1024,
      "optimizer": "Adam",
      "learning_rate": 0.001,
      "hidden": "90/100/40/1",
      "BatchNorm1d": "True",
      "Dropout": "0.2, 0.2",
      "rectifier": "ReLU",
      "epochs": 5,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=5)
wandb.finish()

Training, epoch 1:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.409404551211836, rmse: 9.025695237492545


Training, epoch 2:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 4, loss: 8.940405112016284, rmse: 8.888123406220584


Training, epoch 5:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 5, loss: 8.912951817729974, rmse: 8.880574518571366


test rmse,█▄▃▁▁
train loss,█▃▂▁▁
test rmse,8.88057
train loss,8.91295


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_8", 
      config={
      "batch_size train": 1024,
      "optimizer": "Adam",
      "learning_rate": 0.001,
      "hidden": "90/100/40/1",
      "BatchNorm1d": "True",
      "Dropout": "0.4, 0.2",
      "rectifier": "ReLU",
      "epochs": 5,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=5)
wandb.finish()

Training, epoch 1:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.51230574374988, rmse: 9.04275244753811


Training, epoch 2:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 4, loss: 8.991581451930392, rmse: 8.912133897530767


Training, epoch 5:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 5, loss: 8.965364548080952, rmse: 8.910821822240003


test rmse,█▄▂▁▁
train loss,█▃▂▁▁
test rmse,8.91082
train loss,8.96536


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_9", 
      config={
      "batch_size train": 1024,
      "optimizer": "Adam",
      "learning_rate": 0.001,
      "hidden": "90/300/150/1",
      "BatchNorm1d": "True",
      "Dropout": "0.4, 0.2",
      "rectifier": "ReLU",
      "epochs": 5,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=5)
wandb.finish()

Training, epoch 1:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.34788126438665, rmse: 8.997836771733407


Training, epoch 2:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 4, loss: 8.910571722881967, rmse: 8.876799147505299


Training, epoch 5:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 5, loss: 8.879927865105044, rmse: 8.85182006920246


test rmse,█▅▄▂▁
train loss,█▃▂▁▁
test rmse,8.85182
train loss,8.87993


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_10", 
      config={
      "batch_size train": 1024,
      "optimizer": "Adam",
      "learning_rate": 0.001,
      "hidden": "90/300/150/1",
      "BatchNorm1d": "True",
      "Dropout": "0.4, 0.2",
      "rectifier": "PReLU",
      "epochs": 10,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=10)
wandb.finish()

Training, epoch 1:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.42426381855203, rmse: 9.027336461384385


Training, epoch 2:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 9, loss: 8.78713596628855, rmse: 8.819568580733259


Training, epoch 10:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 10, loss: 8.77311149706912, rmse: 8.801948124090169


test rmse,█▅▄▃▂▂▂▁▂▁
train loss,█▄▃▃▂▂▂▁▁▁
test rmse,8.80195
train loss,8.77311


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_11", 
      config={
      "batch_size train": 1024,
      "optimizer": "Adam",
      "learning_rate": 0.001,
      "hidden": "90/300/150/1",
      "BatchNorm1d": "True",
      "Dropout": "0.4, 0.1",
      "rectifier": "PReLU",
      "epochs": 10,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=10)
wandb.finish()

Training, epoch 1:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.39972308060505, rmse: 9.02901571793043


Training, epoch 2:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 9, loss: 8.770116731437339, rmse: 8.804586956472049


Training, epoch 10:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 10, loss: 8.748870952894892, rmse: 8.798853364383515


test rmse,█▅▄▃▃▂▂▂▁▁
train loss,█▄▃▃▂▂▂▁▁▁
test rmse,8.79885
train loss,8.74887


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_12_", 
      config={
      "batch_size train": 512,
      "optimizer": "Adam",
      "learning_rate": "0.001 + weight_decay=0.0005",
      "hidden": "90/300/150/1",
      "BatchNorm1d": "True",
      "Dropout": "0.4, 0.1",
      "rectifier": "PReLU",
      "epochs": 10,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=10)
wandb.finish()

Training, epoch 1:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.378917017901877, rmse: 9.031019509451715


Training, epoch 2:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 9, loss: 8.907157654045681, rmse: 8.897475588108


Training, epoch 10:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 10, loss: 8.90514558791817, rmse: 8.91474583295577


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_14", 
      config={
      "batch_size train": 1024,
      "optimizer": "Adam",
      "learning_rate": "0.001 + weight_decay=0.0001",
      "hidden": "90/300/150/1",
      "BatchNorm1d": "True",
      "Dropout": "0.4, 0.1",
      "rectifier": "PReLU",
      "epochs": 10,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=10)
wandb.finish()

Training, epoch 1:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.509095424548335, rmse: 9.032046129044415


Training, epoch 2:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 9, loss: 8.772905744116793, rmse: 8.798944289523904


Training, epoch 10:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 10, loss: 8.763241315235446, rmse: 8.798353745803617


test rmse,█▅▃▃▂▂▁▁▁▁
train loss,█▄▃▂▂▂▁▁▁▁
test rmse,8.79835
train loss,8.76324


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_15", 
      config={
      "batch_size train": 512,
      "optimizer": "Adam",
      "learning_rate": "0.001 + weight_decay=0.0001",
      "hidden": "90/300/150/1",
      "BatchNorm1d": "True",
      "Dropout": "0.4, 0.1",
      "rectifier": "PReLU",
      "epochs": 10,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=10)
wandb.finish()

Training, epoch 1:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.388079006894186, rmse: 9.038003521604717


Training, epoch 2:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 9, loss: 8.814512619841649, rmse: 8.82489259714216


Training, epoch 10:   0%|          | 0/906 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 10, loss: 8.804990900432363, rmse: 8.833773848301119


test rmse,█▅▄▃▂▂▂▁▁▁
train loss,█▄▃▂▂▂▁▁▁▁
test rmse,8.83377
train loss,8.80499


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_15", 
      config={
      "batch_size train": 1024,
      "optimizer": "Adam",
      "learning_rate": "0.001 + weight_decay=0.0001",
      "hidden": "90/300/200/100/1",
      "BatchNorm1d": "True",
      "Dropout": "0.4, 0.1, 0.1",
      "rectifier": "PReLU",
      "epochs": 10,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=10)
wandb.finish()

Training, epoch 1:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.441957063775886, rmse: 8.963926214123756


Training, epoch 2:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 9, loss: 8.774795432210997, rmse: 8.769157461605143


Training, epoch 10:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 10, loss: 8.75656816398299, rmse: 8.762377485615607


test rmse,█▅▄▃▃▂▂▂▁▁
train loss,█▄▃▃▂▂▂▁▁▁
test rmse,8.76238
train loss,8.75657


In [ ]:
wandb.init(
      project="hw1_1",
      name="attempt_16", 
      config={
      "batch_size train": 1024,
      "optimizer": "Adam",
      "learning_rate": "0.001",
      "hidden": "90/300/200/100/1",
      "BatchNorm1d": "True + weight_decay=0.0001",
      "Dropout": "0.4, 0.1, 0.1",
      "rectifier": "PReLU",
      "epochs": 15,
      })

wandb.watch(model)
train(model, optimizer, criterion, train_loader, test_loader, y_test, device, n_epochs=15)
wandb.finish()

Training, epoch 1:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 1, loss: 9.428125366978854, rmse: 8.963304527413813


Training, epoch 2:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

...

Epoch: 14, loss: 8.653284230886998, rmse: 8.748351874528465


Training, epoch 15:   0%|          | 0/453 [00:00<?, ?it/s]

  0%|          | 0/202 [00:00<?, ?it/s]

Epoch: 15, loss: 8.641368980228457, rmse: 8.741269514731018


test rmse,█▅▄▄▃▂▂▂▂▂▂▁▁▁▁
train loss,█▅▄▃▃▃▂▂▂▂▂▁▁▁▁
test rmse,8.74127
train loss,8.64137


**Ура! Победа!**

## Задание 2. (0 баллов, но при невыполнении максимум за все задание &mdash; 0 баллов)

Напишите небольшой отчет о том, как вы добились полученного качества: какие средства использовали и какие эксперименты проводили. Подробно расскажите об архитектурах и значениях гиперпараметров, а также какие метрики на тесте они показывали. Чтобы отчет был зачтен, необходимо привести хотя бы 3 эксперимента.

  Итак, это была тяжелая робота для первого раза.

  Прежде всего я была внимательной на лекциях и семинарах. Отнормировала данные и таргет.

  Первая модель состояла из двух полносвязных слоев, функция активации ReLU. Оптимайзер SGD с lr=0.01. Затем уменьшила размер шага в 10 раз, чтоб не прыгать по функции потерь. Увидела улучшение на первых эпохах.

  Добавила еще один слой в модель. Стало намного хуже, но решила не сдаваться и поработать с ними. Добавила батч нормализацию. Обучение модели ускорилось, по графику видно резкое убывание рмсе. Эх, надо было добавить эпох... Но я пошла дальше накручивать качество. 

  Решила воспользоваться алгоритмом оптимизации Адама вместо обычного стохастического ГД. Так как Адам объединяет лучшие свойства алгоритмов AdaGrad и RMSProp, качество взлетело мгновенно. Попробовала поиграть с размером батча. 
  Решила добавить дропауты между слоями для уменьшения переобучения. Поэксперементировала с вероятностью исключения. Увеличила ширину слоев. Качество улучшилось. 

  Заменила функцию активации на PReLU, которая дает нейронам возможность выбирать лучший уклон в отрицательной области. 

  Поработала с L2 регуляризацией адама, небольшое улучшение. Добавила еще один слой. Добилась целевого качества.

   Итоговая модель написана в коде, результат ее работы последний приведенный в экспериментах. Графики и параметры моделей лежат на wandb. Вот ссылочка,если что: https://wandb.ai/liz04ka/hw1_1?workspace=user-liz04ka. Понравилось приложение, тоже всем советую, очень удобно.

   Особую благодарность за успешное выполнение работы выражаю 4м полносвязным слоям, батч-нормализации, дропаутам, prelu и, конечно же, Адаму.



